# Genetic Algorithm for Continuous Function Optimization

The goal is to maximize a continuous nonlinear objective function of two variables $x$ and $y$ defined on a bounded domain $[0, 2\pi]\times[0, 2\pi]$ 

The optimized function was defined as: $$f(x,y)=|\sin x + \sin 2x + \sin 4x + \sin 8x | + |\cos y + \cos 2y + \cos 4y + \cos 8y |$$

In [7]:
import numpy as np
import random

# This is not used for the algorithm, only to verify solution later 
from scipy.optimize import differential_evolution

In [2]:
def f_xy(x, y):
    """
    f(x,y) = |sin x + sin 2x + sin 4x + sin 8x| + |cos y + cos 2y + cos 4y + cos 8y|
    Maximize, with 0 <= (x,y) <= 2pi
    """
    sx = np.sin(x) + np.sin(2*x) + np.sin(4*x) + np.sin(8*x)
    cy = np.cos(y) + np.cos(2*y) + np.cos(4*y) + np.cos(8*y)
    return np.abs(sx) + np.abs(cy)

In [3]:
def init_population(rng, pop_size=1000):
    return rng.uniform(0.0, 2*np.pi, size=(pop_size, 2))

def crossover(pop, rng):
    perm = rng.permutation(len(pop)).reshape(-1, 2)
    p1, p2 = pop[perm[:, 0]], pop[perm[:, 1]]

    c1 = np.column_stack([p1[:, 0], 0.5 * (p1[:, 1] + p2[:, 1])])
    c2 = np.column_stack([0.5 * (p1[:, 0] + p2[:, 0]), p2[:, 1]])

    return np.vstack([c1, c2])

Mutation is applied to each offspring by randomly selecting one coordinate ($x$ or $y$) and modifying it using a small multiplicative factor (0.999 or 1.001). Mutations that produce values outside the interval $[0, 2\pi]$ are rejected. 

Parent and offspring populations are merged and tournament selection is used to construct the next generation by selecting the better individual from randomly formed pairs. The algorithm iteratively applies crossover, mutation and selection and returns the best individual found.


In [4]:
def mutate(children, rng, factors=(0.999, 1.001)):
    pop = children.copy()
    n = len(pop)

    dim = rng.integers(0, 2, size=n)
    factor = rng.choice(factors, size=n)

    for i in range(n):
        d = dim[i]
        new_val = pop[i, d] * factor[i]
        if 0.0 <= new_val <= 2*np.pi:
            pop[i, d] = new_val

    return pop


def tournament_selection(parents, children, rng):
    pool = np.vstack([parents, children])
    N = len(parents)

    a = rng.integers(0, len(pool), size=N)
    b = rng.integers(0, len(pool), size=N)

    fa = f_xy(pool[a, 0], pool[a, 1])
    fb = f_xy(pool[b, 0], pool[b, 1])

    return pool[np.where(fa >= fb, a, b)]


def run_genetic_algorithm(n_iters=300, seed=None):
    rng = np.random.default_rng(seed)
    pop = init_population(rng)

    for _ in range(n_iters):
        children = crossover(pop, rng)
        children = mutate(children, rng)
        pop = tournament_selection(pop, children, rng)

    values = f_xy(pop[:, 0], pop[:, 1])
    best = np.argmax(values)

    return pop[best], values[best]

Run 300 iterations and repeate 10 times with different random seeds.

In [5]:
results = []
for i in range(10):
    (x, y), val = run_genetic_algorithm(n_iters=300, seed=i)
    results.append(val)
    print(f"run {i+1}: f={val:.6f}")

print("Best overall:", max(results))


run 1: f=6.468462
run 2: f=6.309330
run 3: f=6.462234
run 4: f=6.309330
run 5: f=6.489513
run 6: f=6.410775
run 7: f=6.308862
run 8: f=6.309331
run 9: f=6.444113
run 10: f=6.309330
Best overall: 6.4895134577245805


The results indicate stable convergence of the genetic algorithm, with small differences between runs caused by randomness. Additionally, to verify the result, run differential evolution from the SciPy library.

In [6]:
def neg_f(v):
    x, y = v
    return -f_xy(x, y)

bounds = [(0.0, 2*np.pi), (0.0, 2*np.pi)]
res = differential_evolution(neg_f, bounds, seed=0, polish=True)
x_star, y_star = res.x
print("best_f:", -res.fun)
print("x,y:", x_star, y_star)


best_f: 6.494310414845856
x,y: 6.011472750238535 6.283185307179586
